# CS375 - Assignment 1
#### Group 6 

## Part 1: ImageNet training and testing

In [1]:
%matplotlib inline 
import os
import numpy as np
import tensorflow as tf
import pymongo as pm
import gridfs
import cPickle
import scipy.signal as signal
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook, trange

### Getting data from the database
Let's connect to the database and pull the data training and test data that is stored while training our network.
In order to find the right experiment id, it is useful to display which experiments are stored in the database first.

In [2]:
# connect to database
dbname = 'imagenet'
collname = 'alexnet' # TODO need to iterate for other nets in pt. 1
port = 24444

conn = pm.MongoClient(port = port)
coll = conn[dbname][collname + '.files']

# print out saved experiments in collection coll
print(coll.distinct('exp_id'))

[u'experiment_1']


### Plotting the training curve
To see the training curve for the experiment, we pull the training loss and plot it, along with a smoothed version of the training loss obtained by convolving the loss with a boxcar.

In [ ]:
def get_losses(exp_id):
    """
    Gets all loss entries from the database and concatenates them into a vector
    """
    q_train = {'exp_id' : exp_id, 'train_results' : {'$exists' : True}}
    return np.array([_r['loss'] 
                     for r in coll.find(q_train, projection = ['train_results']) 
                     for _r in r['train_results']])


def get_boxcar(num_pts, total_length):
    my_ones = np.ones(num_pts)/num_pts
    return np.hstack([my_ones])

ydict = {}
ydict['losses'] = get_losses('experiment_1')
smoothing_num = 100;
my_box = get_boxcar(smoothing_num, losses.shape[0])
ydict['smooth' + str(smoothing_num)] = signal.convolve(ydict['losses'],my_box)

plt.plot(ydict['losses'], color="blue")
plt.xlabel("Iteration")
plt.ylabel("Training Loss")
plt.title("Experiment 1: Training Loss vs. Iteration")
plt.grid()


In [ ]:
plt.plot(ydict['smooth' + str(smoothing_num)], color="blue")
plt.xlabel("Iteration")
plt.ylabel("Training Loss")
plt.title("Experiment 1: Smoothed Training Loss vs. Iteration")
plt.grid()


Smoothed plot of training loss. Please note that the bump at around iteration 80,000 is due to having to recover the model and resume training after a gcloud crash.

### Plotting the validation results
After our train_imagenet function has evaluated the Image Net validation set for the time we can have a look at the validation results of for example exp_id='experiment_1'. Again, you need to pull the validation data from the database first. The validation data consists of the top1 and top5 accuracy that you have implemented previously. 

We have provided a function that pulls the necessary data from the database. Your task is to plot the validation curve of the top1 and top5 accuracy. Label the graphs respectively and describe what you see.

In [ ]:
def get_validation_data(exp_id):
    """
    Gets the validation data from the database (except for gridfs data)
    """
    q_val = {'exp_id' : exp_id, 'validation_results' : {'$exists' : True}, 'validates' : {'$exists' : False}}
    val_steps = coll.find(q_val, projection = ['validation_results'])
    top1 = [val_steps[i]['validation_results']['topn_val']['top1'] 
            for i in range(val_steps.count())]
    top5 = [val_steps[i]['validation_results']['topn_val']['top5'] 
            for i in range(val_steps.count())]
    return top1, top5

### PLOT VALIDATION RESULTS HERE
top1, top5 = get_validation_data('experiment_1')
fig1 = plt.figure()
ax1 = fig1.add_subplot(111)
plt.xlabel("Step")
plt.ylabel("Accuracy")
plt.title("Experiment 1 - Top1: Accuracy vs. Step")
plt.grid()
ax1.plot(top1)

fig2 = plt.figure()
ax2 = fig2.add_subplot(111)
plt.xlabel("Step")
plt.ylabel("Accuracy")
plt.title("Experiment 1 - Top5: Accuracy vs. Step")
plt.grid()
ax2.plot(top5)


## Neural analysis
Now let's move on to fetching and displaying the computations and tests that you ran in the neural_analysis agg_func. We will first pull the data which was stored in the validation results in the database. Therefore, we call the function 'get_neural_validation_data'. The returned 'validation_data' is a list of dictionaries that contain your validation results. It's keys contain amongst others the 'exp_id' of your experiment, the evaluated 'step', and the actual 'validation_results'. In part 2 of this assignment, you should have evaluated your model at different iteration steps.

In the following, your task will be to plot your evaluations at different iteration steps. Therefore, index the data list with the appropriate indices, plot the results and mark all of the following plots with the iteration step.

In [ ]:
def get_neural_validation_data(exp_id):
    

In [5]:
exp_id = 'experiment_1'
q_val = {'exp_id' : exp_id, 'validation_results' : {'$exists' : True}, 'validates': {'$exists' : True}}
val_steps = coll.find(q_val, projection = ['validation_results', 'validates', 'exp_id'])

In [8]:
results = [val_steps[i] for i in range(val_steps.count())]

In [26]:
coll.find({'_id':results[0]['validates']})[0]['step']

{u'_id': ObjectId('59e09bc0d4fdab1c6991748d'),
 u'_saver_num_data_files': 1,
 u'_saver_write_version': 2,
 u'chunkSize': 261120,
 u'duration': 1.7959988117218018,
 u'exp_id': u'experiment_1',
 u'filename': u'/home/cocjinjb/.tfutils/localhost:24444/imagenet/alexnet/experiment_1/checkpoint-60000.tar',
 u'length': 912128000,
 u'md5': u'88ea4b5fd92f30b824aa297a3f4e1a57',
 u'params': {u'dont_run': False,
  u'inter_op_parallelism_threads': 40,
  u'learning_rate_params': {u'boundaries': [150000, 300000, 450000],
   u'func': {u'active_branch': u'john',
    u'active_branch_in_origin': True,
    u'clean': False,
    u'commit': u'afaf5719c9b6c8758b63c086e654076e881d4f5d',
    u'commit_in_log': True,
    u'git_dir': u'/home/cocjinjb/cs375/.git',
    u'modname': u'__main__',
    u'objname': u'piecewise_constant_wrapper',
    u'remote_urls': [u'https://github.com/j-abc/cs375.git'],
    u'source_path': u'train_imagenet.py'},
   u'values': [0.01, 0.005, 0.001, 0.0005]},
  u'load_params': {u'collname':

In [27]:
# for res in results:
#     res['step'] = coll.find({'_id': res['validates']})[0]['step']

IndexError: no such item for Cursor instance

In [44]:
results = [val_steps[i] for i in range(val_steps.count())]
i = 0
for res in results:
    try:
        res['step'] = coll.find({'_id': res['validates']})[0]['step']    
        print(res['step'])
    except:
        res['step'] = -1
        print(i)
    i = i + 1

60000
120000
120000
120000
180000
180000
180000
180000
180000
180000
10
11
12
210000
210000
210000
210000
210000


In [45]:
len(results)

18

In [3]:
    return results

validation_data = get_neural_validation_data(exp_id='experiment_1')
print('data keys:')
print(validation_data[0].keys())

IndexError: no such item for Cursor instance

In [29]:

def get_neural_validation_data(exp_id):
    q_val = {'exp_id' : exp_id, 'validation_results' : {'$exists' : True}, 'validates': {'$exists' : True}}
    val_steps = coll.find(q_val, projection = ['validation_results', 'validates', 'exp_id'])

    results = [val_steps[i] for i in range(val_steps.count())]
    for res in results:
        if 'step' in res:
            res['step'] = coll.find({'_id': res['validates']})[0]['step']
        else:
            res['step'] = 'blah'
    return results

validation_data = get_neural_validation_data(exp_id='experiment_1')
print('data keys:')
print(validation_data[0].keys())

data keys:
[u'exp_id', u'_id', u'validates', u'validation_results']


In [ ]:
coll.find()

In [46]:
import dldata

Can't import separate mcc package
Can't import asgd.
Can't import scikits stuff


/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [ ]:
dldata.metrics.utils.compute_metric_base

In [41]:
"""
You will need to EDIT this part.
Please subselect 'validation_data' with 'idx' to pick the data entry for your desired iteration step.
1.) Assign 
    data = 'validation_data[idx]['validation_results']['valid0']'
and
    step = validation_data[idx]['step']
2.) Choose the target_layers you want to evaluate on.
"""
# TODO FIX THIS OH MY GOD
#target_layers = ['pool1', 'conv2', 'conv3', 'conv4', 'conv5', 'pool5', 'fc6', 'fc7']
target_layers = ['pool1']

# TODO This needs to include the data at 0 as well
for idx in range(len(validation_data)):
    data = validation_data[idx]['validation_results']['valid0']
    if 'pleasework_pool1' in data.keys():
         print(data['pleasework_pool1'])
#     print data.keys()
#print(data.keys())
#print(data['continuous_pool1'])
#print(data['within_categorization_pool1'])

{u'corr_splitarray_loss': [0.503497395057094, 0.47219189166185205, 0.5214176860125014, 0.4667699138448248, 0.5416233678116966, 0.49569317278469893, 0.5537914006199944, 0.5200320734259618, 0.4372126680121513, 0.489877605804634, 0.5566503502405833, 0.5161707360106444, 0.5145740052313867, 0.5215274551153803, 0.4569213445384902, 0.5893115077432984, 0.5358845773749783, 0.4695785452697411, 0.532088778686111, 0.468479119250375], u'rsquared_loss_stderror': 0.054668133953178884, u'rsquared_splitarray_loss': [0.7627983734791657, 0.7324496212841523, 0.7913588066061612, 0.7170270861587661, 0.8086459796916369, 0.7771753628108867, 0.8050121214553344, 0.798186297699666, 0.6880841602306444, 0.783089477273796, 0.8489316042252322, 0.8091481397670103, 0.7709406811976056, 0.7985779172242242, 0.7364440837764913, 0.9301874424360556, 0.8238852783733291, 0.7293731352547298, 0.8673767436427591, 0.7473447381699297], u'corr_loss_stderror': 0.037630526232295805, u'rsquared_loss': 0.7863018525378789, u'corr_loss':

In [34]:
data.keys()

[u'conv1_kernel', u'pleasework_pool1']

In [ ]:
len(validation_data)

In [ ]:
def get_data(idx):
    return validation_data[idx]['validation_results']['valid0']

In [ ]:
data = get_data(9)

In [ ]:
data.keys()

In [ ]:
step = validation_data[idx]['step']

### Analyzing the RDM results

We will start with visualizing the RDMs you computed for each layer and the IT neurons. 

Please implement 'plot_rdms' that visualizes the rdms of all target_layers and displays the layer name as well as the spearman_corrcoef and the iteration step in the plot title.


In [ ]:
def plot_rdms(data, target_layers, step):
    """
    Displays the RDMs of all target layers.
    
    You will need to EDIT this part. Please implement a function that visualizes 
    the rdms of all target_layers and displays the layer name as well as the 
    spearman_corrcoef in the plot title.
    """
    for i, layer in enumerate(target_layers):
        rdm = data['rdm_%s' % layer]
        spearman_corrcoef = data['spearman_corrcoef_%s' % layer]
        ### YOUR CODE HERE
        fig = plt.figure(figsize=(8, 8))
        m = fig.gca().matshow(rdm)
        plt.colorbar(m)
        plt.title(str(layer) + ": spearman_corr_coeff = " + str(spearman_corrcoef) + "; step: " + str(step))
        
        ### END OF YOUR CODE

plot_rdms(data, target_layers, step)

### Analyzing the categorization results

Now let's visualize the results of the categorization test we performed for each layer. 

Your task is to plot the confusion matrix 'cms' across all categorized classes for each layer. You will need 'cms', 'labelset' and 'accbal' from data['categorization_"insert_target_layer_here"']['result_summary']. Label the x and y axis with 'labelset'. Put the layer name, average accuracy and the iteration step in the plot title. To get the average accuracy substract 0.5 from the mean of 'accbal' and multiply the result with 2. Describe what you see.

In [ ]:
def plot_categorization_results(data, target_layers, step, category=None):
    """
    Plots the confusion matrix and the average classification accuracy for each layer.
    
    You will need to EDIT this section.
    """
    for i, layer in enumerate(target_layers):
        if category:
            k = 'within_categorization_%s' % layer                
            try:
                categorization_results = data[k][category]['result_summary']
            except:
                raise Exception(category + " is not a real category in data...")
        else:
            k = 'categorization_%s' % layer
            categorization_results = data[k]['result_summary']
        ### YOUR CODE HERE
        fig = plt.figure(figsize=(8, 8))
        m = fig.gca().matshow(np.array(categorization_results['cms']).mean(2))
        plt.xticks(range(8), categorization_results['labelset'])
        plt.yticks(range(8), categorization_results['labelset'])
        plt.colorbar(m)
        ave_acc = (np.mean(categorization_results['accbal']) - .5) * 2
        if category:
            plt.title("category: " + category + " , " + k + "\n: mean_accuracy = " + str(ave_acc) + ", iteration_step = " + str(step))
        else:
            plt.title(k + ": mean_accuracy = " + str(ave_acc) + "\n, iteration_step = " + str(step))

        m.set_cmap('jet')
        ax = plt.gca()
        ax.xaxis.tick_bottom()
        
        
        ### END OF YOUR CODE
# Plot across categorization
#plot_categorization_results(data, target_layers, step)

# Plot within categorization
categories = ['Animals', 'Boats', 'Cars', 'Chairs', 'Faces', 'Fruits', 'Planes', 'Tables']
for category in categories:
    plot_categorization_results(data, target_layers, step, category=category)


### Analyzing the regression results

We will now display the results of the regression test. 

Please print (1 - data['it\_regression_"insert_target_layer_here"']['noise_corrected_multi_rsquared_loss']) for each layer and step and label the print out with the layer name and step. Describe what you observe.

In [ ]:
def plot_regression_results(data, target_layers, step):
    """
    Prints out the noise corrected multi rsquared loss for each layer.
    
    You will need to EDIT this function.
    """
    for layer in target_layers:
        k = 'it_regression_%s' % layer
        regression_results = data[k]
        ### YOUR CODE HERE
        calculated_regression_val = 1 - regression_results['noise_corrected_multi_rsquared_loss']
        print('step: ' + str(step) + ", layer: " + layer + ", calculated_value: " + str(calculated_regression_val))
        ### END OF YOUR CODE

plot_regression_results(data, target_layers, step)

### Visualizing the conv1 kernels
Lastly, we will plot the conv1 filter kernels. 

Your task is to visualize the 'conv1_kernel' in a 12 x 8 grid. Therefore, you just have to plot each kernel as an image in a subplot. The kernels are stored in data['conv1_kernel']. Print out the iteration step and plot the kernels and describe what you see.

In [ ]:
def plot_conv1_kernels(data, step):
    """
    Plots the 96 conv1 kernels in a 12 x 8 subplot grid.
    
    You will need to EDIT this function.
    """
    print('Iteration step: %d' % step)
    kernels = np.array(data['conv1_kernel'])
    ### YOUR CODE HERE
    x = 12
    y = 8
    f, axarr = plt.subplots(x, y, figsize=(4*x, 4*y))
    for i in range(x):
        for j in range(y):
            m = axarr[i,j].imshow(kernels[:,:,:,i*y + j])
            axarr[i,j].get_xaxis().set_visible(False)
            axarr[i,j].get_yaxis().set_visible(False)
    f.subplots_adjust(hspace=0)
    f.subplots_adjust(wspace=0)

    ### END OF YOUR CODE
           
plot_conv1_kernels(data, step)

### TODO WRITE DESCRIPTION OF THIS PLOT and all plots...